# 03 – Estimation (MLE with Monte-Carlo marginalisation & Stan skeleton)

In this notebook, we fit the mixed hidden Markov model to a simulated dataset
(`data/simulated/ref_scenario.csv`) generated in `02_simulator.ipynb`.

It contains:
- A Monte-Carlo marginalization approach for MLE (approximate marginal likelihood)
- A CmdStanPy / Stan model skeleton for Bayesian inference (marginalize discrete states via forward algorithm)

**Notes:**
- MLE via MC is computationally heavy (many forward calls); we provide a small quick test and an option to scale up.
- The Stan skeleton is illustrative and requires more tuning & priors to run for production.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from scipy.stats import multivariate_normal
from tqdm.notebook import tqdm

import time
import pickle
import os

from models.emissions import EmissionModel
from models.transitions import TransitionModel

Stable log-sum-exp and forward using EmissionModel.logpdf  (check eqtn 11 in the paper):

In [ ]:
EPS = 1e-12

def logsumexp_arr(a):
    m = np.max(a)
    return m + np.log(np.sum(np.exp(a-m)))

def forward_loglik_subject(obs, times, init_probs, trans_mat, emission_model, g):

    """Compute the log-likelihood of a single subject given fixed
    individual effects g using the forward algorithm in log-space for numerical stability.
    Args:
        obs (np.ndarray): Observations for the subject (T x D).
        times (np.ndarray): Observation times for the subject (T,).
        g: dict of random effects for the subject.
        """
    T = len(obs)
    n_states = 2
    #initialize log alpha
    log_alpha = np.zeros((T, n_states)) # log forward probabilities

    for s in range(n_states):
        log_em = emission_model.logpdf(obs[0], g, times[0], s)
        log_alpha[0, s] = np.log(init_probs[s] + EPS) + log_em 

    #normalize in log-space by subtracting logsumexp
    for t in range(1, T): #forward recursion
        new_log_alpha = np.full(n_states, -np.inf)
        for j in range(n_states):
            #compute logsum over previous states
            prev = log_alpha[t-1, :] + np.log(trans_mat[:, j] + EPS) 
            s_prev = logsumexp_arr(prev)
            log_em = emission_model.logpdf(obs[t], g, times[t], j)
            new_log_alpha[j] = s_prev + log_em #combine transition and emission log-probs
        log_alpha[t, :] = new_log_alpha 
    
    return logsumexp_arr(log_alpha[-1, :]) #marginal log-likelihood is logsumexp of final alphas  

Load simulated data and prep per-subject datasets

In [ ]:
data_path = '../data/simulated/ref_scenario.csv'
assert os.path.exists(data_path), f"Data file not found at {data_path}. Run 02_simulator.ipynb to generate it."

df = pd.read_csv(data_path)
df.head()

In [ ]:
subjects = df['ID'].unique()
subj_data = {}

for sid in subjects:
    sub = df[df.ID ==sid].sort_values("Week")
    obs = sub[["FEV1","PRO"]].values
    times = sub["Week"].values.astype(float)
    subj_data[int(sid)] = {'obs': obs, 'times': times}

len(subj_data), list(subj_data.keys())[:5] # Check number of subjects and first 5 IDs

Monte_Carlo marginal log-likelihood per subject

-For each candidate population parameter set, for each subject we:

-> draw K samples of g ~ N(0, x2_*) using the candidate x2 variances,

-> compute forward log-likelihood for each g,

-> average the likelihoods (in log-space using log-sum-exp) to approximate the marginal likelihood.

Since this is slow for many subjects / large K, it's advisable to use small tests (e.g., K=30) before scaling.

In [ ]:
def sample_g_matrix_em_params(n_samples, rng, x2_params):
    """
    Draw n_samples of g according to x2 variances dict.
    x2_params: dict with keys ['x2_FEV1R','x2_FEV1E','x2_PROR','x2_PROE']
    Returns list of dicts length n_samples.
    """
    s_FEv1R = np.sqrt(x2_params['x2_FEV1R'])  #standard deviations
    s_FEV1E = np.sqrt(x2_params['x2_FEV1E'])
    s_PROR = np.sqrt(x2_params['x2_PROR'])
    s_PROE = np.sqrt(x2_params['x2_PROE'])
    rng = np.random.default_rng(rng)
    gs = []
    for _ in range(n_samples): #draw n_samples
        g = {
            "gFEV1R": float(rng.normal(0.0, s_FEv1R)),
            "gFEV1E": float(rng.normal(0.0, s_FEV1E)),
            "gPROR":  float(rng.normal(0.0, s_PROR)),
            "gPROE":  float(rng.normal(0.0, s_PROE))
        }
        gs.append(g)
    return gs


In [ ]:
def approx_subject_log_marginal(obs, times, init_probs, trans_mat, em_params, K=30, rng_seed=None):
    """Approximate the marginal log-likelihood of a subject
    by Monte-Carlo integration over g.
    Args:
        obs (np.ndarray): Observations for the subject (T x D).
        times (np.ndarray): Observation times for the subject (T,).
        em_params: dict of emission parameters including variance components.
        K (int): Number of Monte-Carlo samples."""

    #build emission model from em_params
    em = EmissionModel(
        hFEV1R=em_params['hFEV1R'],
        hFEV1E=em_params['hFEV1E'],
        x2_FEV1R=em_params['x2_FEV1R'],
        x2_FEV1E=em_params['x2_FEV1E'],
        hPROR=em_params['hPROR'],
        hPROE=em_params['hPROE'],
        x2_PROR=em_params['x2_PROR'],
        x2_PROE=em_params['x2_PROE'],
        r2_FEV1=em_params['r2_FEV1'],
        r2_PRO=em_params['r2_PRO'],
        qR=em_params['qR'],
        qE=em_params['qE'],
        PE=em_params['PE'],
        PHL=em_params['PHL']
    )

    #sample K g's
    x2_params = {k: em_params[k] for k in ['x2_FEV1R','x2_FEV1E','x2_PROR','x2_PROE']} 
    gs = sample_g_matrix_em_params(K, rng_seed, x2_params)
    #compute log-likelihoods for each g
    logls = []
    for g in gs:
        logl_g = forward_loglik_subject(obs, times, init_probs, trans_mat, em, g)
        logls.append(logl_g)
    #combine via log-mean-exp: log(1/k * sum exp(logl_g ))
    logls = np.array(logls)
    lm = logsumexp_arr(logls) - np.log(K)
    return lm

## Full data negative log-likelihood to optimize

We assemble a function neg_loglik(params) that maps a parameter vector to the negative log-likelihood across subjects using the Monte Carlo approximation.
For clarity we will optimize a reduced parameter set in this notebook

Examples of parameters we estimate here:

-> hFEV1R, hFEV1E, hPROR, hPROE (modes)

-> log of IIV variances: log_x2_FEV1R, log_x2_FEV1E, log_x2_PROR, log_x2_PROE

-> log of residual variances: log_r2_FEV1, log_r2_PRO

-> Fisher-transformed correlations btn the variables: atanh(qR), atanh(qE)

-> hpRE and hpER (we optimize them on logit scale)

For a quick test we only fit a subset (e.g., hFEV1R, hPROR, log residuals). The fixed_mask is for fixing parameters if desired.

In [ ]:
def logistic(x):
    return 1.0 / (1.0 + np.exp(-x))

def logit(p):
    p = np.clip(p, EPS, 1.0 - EPS)
    return np.log(p / (1.0 - p))
# Parameter indexing helpers (order)
param_names = [
    'hFEV1R','hFEV1E','hPROR','hPROE',
    'log_x2_FEV1R','log_x2_FEV1E','log_x2_PROR','log_x2_PROE',
    'log_r2_FEV1','log_r2_PRO',
    'atanh_qR','atanh_qE',
    'logit_hpRE','logit_hpER'
]

def pack_params_from_vector(vec):
    """
    Convert unconstrained vector to em_params dict and transition params.
    `vec` must follow the param_names order.
    """
    d = {}
    idx = 0
    d['hFEV1R'] = vec[idx]; idx+=1  #must be positive to 
    d['hFEV1E'] = vec[idx]; idx+=1
    d['hPROR']  = vec[idx]; idx+=1
    d['hPROE']  = vec[idx]; idx+=1

    d['x2_FEV1R'] = np.exp(vec[idx]); idx+=1
    d['x2_FEV1E'] = np.exp(vec[idx]); idx+=1
    d['x2_PROR']  = np.exp(vec[idx]); idx+=1
    d['x2_PROE']  = np.exp(vec[idx]); idx+=1

    d['r2_FEV1'] = np.exp(vec[idx]); idx+=1
    d['r2_PRO']  = np.exp(vec[idx]); idx+=1

    d['qR'] = np.tanh(vec[idx]); idx+=1
    d['qE'] = np.tanh(vec[idx]); idx+=1

    d['hpRE'] = logistic(vec[idx]); idx+=1
    d['hpER'] = logistic(vec[idx]); idx+=1

    # placeholders for PE, PHL (fixed)
    d['PE'] = 0.2
    d['PHL'] = 10.0
    return d

# default initial vector set to values close to simulation
init_vec = np.array([
    3.0, 0.5,  # hFEV1R, hFEV1E
    2.5, 0.5,  # hPROR, hPROE
    np.log(0.03), np.log(0.03), np.log(0.09), np.log(0.09),  # log x2
    np.log(0.015), np.log(0.05),  # log r2
    np.arctanh(-0.33), np.arctanh(-0.33),  # atanh q
    logit(0.1), logit(0.3)  # logit hpRE, hpER
])

# Choose a subset to optimize for a quick demo (e.g., first 10 params)
free_idx = list(range(len(init_vec)))  # by default free all
# for a faster demo you might limit:
# free_idx = list(range(10))  # fit only the first 10 parameters

def neg_loglik_mc(vec, subj_data_local, init_probs, trans_mat_fn, K=30, rng_seed=42):
    """
    Negative log-likelihood across all subjects approximated by MC integration.
    trans_mat_fn: function that given em_params returns transition matrix (we keep hpRE/hpER in params)
    """
    em_params = pack_params_from_vector(vec)
    # build trans_mat from hpRE/hpER (no gpRE/gpER here)
    tm = TransitionModel(hpRE=em_params['hpRE'], hpER=em_params['hpER'])
    trans_mat = tm.transition_matrix()

    total_logl = 0.0
    rng_seq = np.random.SeedSequence(rng_seed)
    # Loop subjects (consider batching / parallelization later)
    for i, sid in enumerate(subj_data_local.keys()):
        d = subj_data_local[sid]
        lm = approx_subject_log_marginal(
            d['obs'], d['times'],
            init_probs, trans_mat, em_params, K=K, rng_seed=rng_seq.spawn(1)[0].entropy
        )
        total_logl += lm
    return -float(total_logl)


Quick test of the fn neg_loglik with a small subset 

In [ ]:
test_subjects = {k: subj_data[k] for k in list(subj_data.keys())[:5]}  # first 5 subjects for quick test
init_probs = np.array([0.9, 0.1])  

t0 = time.time()
nl = neg_loglik_mc(init_vec, test_subjects, init_probs, None, K=20, rng_seed=1)
t1 = time.time()
print(f"Neg log-likelihood (5 subjects, K=20): {nl:.3f} computed in {t1 - t0:.1f} seconds")


# Optimization demo

-> We minimize the negative log-likelihood of our mHMM model to find the params that maximizie the likelihood of our observed data.

-> We use L-BFGS-B algorithm - gradient-based optimization algorithm that efficiently handles large numbers of parameters and parameter bounds (e.g. variances > 0). More stable than basic gradient-descent.

In [ ]:
subject_subset = {k: subj_data[k] for k in list(subj_data.keys())[10]}  

#show progress 
history = []
def cb(xk):
    history.append(xk.copy())
    print('.', end='') 

res = minimize(
    fun = lambda v: neg_loglik_mc(v, subject_subset, init_probs, None, K=30, rng_seed=1),
    x0=init_vec,
    method='L-BFGS-B',
    options={'disp': True, 'maxiter': 50}
)

print("\nOptimization result:")
print(res)

# Interpret & decode Viterbi using a point estimate

In [ ]:
if res.success:
    fitted_params = pack_params_from_vector(res.x)
    tm_hat = TransitionModel(hpRE=fitted_params['hpRE'], hpER=fitted_params['hpER'])
    trans_mat_hat = tm_hat.transition_matrix()
    em_hat = EmissionModel(
        hFEV1R=fitted_params['hFEV1R'], hFEV1E=fitted_params['hFEV1E'],
        x2_FEV1R=fitted_params['x2_FEV1R'], x2_FEV1E=fitted_params['x2_FEV1E'],
        hPROR=fitted_params['hPROR'], hPROE=fitted_params['hPROE'],
        x2_PROR=fitted_params['x2_PROR'], x2_PROE=fitted_params['x2_PROE'],
        r2_FEV1=fitted_params['r2_FEV1'], r2_PRO=fitted_params['r2_PRO'],
        qR=fitted_params['qR'], qE=fitted_params['qE'],
        PE=fitted_params['PE'], PHL=fitted_params['PHL']
    )

    from models.mhmm_forward import viterbi  # or we can use the viterbi function from 01_model_math (conv to .py file)
    # Naive g=0
    g0 = {"gFEV1R":0.0,"gFEV1E":0.0,"gPROR":0.0,"gPROE":0.0}
    sample_sid = list(subject_subset.keys())[0]
    sdata = subject_subset[sample_sid]
    states_decoded = viterbi(sdata['obs'], np.array([0.9,0.1]), trans_mat_hat, em_hat, g0, sdata['times'])
    print("Decoded states (naive g=0):", states_decoded)


# Stan Data Builder


In [ ]:
data_path = '../data/simulated/ref_scenario.csv'
assert os.path.exists(data_path), f"Data file not found at {data_path}. Run 02_simulator.ipynb to generate it."

df = pd.read_csv(data_path)
subjects = sorted(df['ID'].unique())
N = len(subjects)

y1_flat, y2_flat, time_flat= [], [], []
subj_start, subj_len, trt_slp = [], [], []

pos = 0 #position in flat arrays
for sid in subjects:
    sub = df[df.ID == sid].sort_values("Week")
    subj_start.append(pos + 1)  #1-based index for Stan 
    L = sub.shape[0]
    subj_len.append(L)
    pos += L

    y1_flat.extend(sub['FEV1'].values.tolist())
    y2_flat.extend(sub['PRO'].values.tolist())
    time_flat.extend(sub['Week'].values.tolist())

    trt_slp.append(0.0) 

stan_data = {
    'N': N,
    'T_max': max(subj_len),
    'total_obs': len(y1_flat),
    'y1': y1_flat,
    'y2': y2_flat,
    'time_flat': time_flat,
    'subj_start': subj_start,
    'subj_len': subj_len,
    'trt_slp': trt_slp,
    'init_prob': [0.9, 0.1]
}

print(f'Stan data built for {N} subject, total{len(y1_flat)} observations.')
print(f'Example: subj_len[:5] = {'subj_len[:5]}'}')

Run CmdStanPy Sampling

In [ ]:
from cmdstanpy import CmdStanModel
import time, pickle 

stan_file = '../models/stan/mhmm_marginal.stan'

assert os.path.exists(stan_file), f"Stan model file not found at {stan_file}. Check models/stan directory."

model = CmdStanModel(stan_file=stan_file)

#Run CmdStanPy Sampling
start = time.time()
fit = model.sample(
    data=stan_data,
    seed=123,
    chains=4,
    parallel_chains=4,
    iter_warmup=1000,
    iter_sampling=1000,
    adapt_delta=0.85,
    show_progress=True 
)
end = time.time()
print(f'Sampling completed in {end - start/60:.2f} minutes.')

os.makedirs('../data/results', exist_ok=True)
fit.save_csvfiles(dir='../data/results')
with open('../data/results/mhmm_stan_fit.pkl', 'wb') as f:
    pickle.dump(fit, f)

print("Stan fit results saved.") 

# Posterior Summaries and Diagnostics

In [ ]:
import arviz as az 

fit_path = '../data/results/mhmm_stan_fit.pkl'
assert os.path.exists(fit_path), f"Fit file not found at {fit_path}. Load the fit results first."

with open(fit_path, 'rb') as f:
    fit = pickle.load(f)

idata = az.from_cmdstanpy(posterior=fit)

params_of_interest = [
    'hFEV1R','hFEV1E','hPROR','hPROE',
    'r2_FEV1','r2_PRO',
    'qR','qE', 'hpRE','hpER',
    'PE','PHL'] 

#we convert transformed params back to original scale for variances
post = fit.draws_pd() 
post['hpRE'] = 1 / (1 + np.exp(-post['logit_hpRE']))
post['hpER'] = 1 / (1 + np.exp(-post['logit_hpER']))
post['qR'] = np.tanh(post['atanh_qR'])
post['qE'] = np.tanh(post['atanh_qE'])

summary = post[params_of_interest].describe(percentiles=[0.025, 0.5, 0.975]).T
summary = summary[['mean', 'std', '2.5%', '50%', '97.5%']]
print("Posterior summary (mean +-SD, 95% CrI):")
display(summary.round(3))


#convergence diagnostics
az.plot_trace(idata, var_names=['hFEV1R','hPROR','r2_FEV1','hpRE'], compact=True)
plt.tight_layout()
plt.show()

#compare to reference values
ref_params = {
    'hFEV1R': 3.0,
    'hFEV1E': 0.5,
    'hPROR': 2.5,
    'hPROE': 0.5,
    'r2_FEV1': 0.015,
    'r2_PRO': 0.05,
    'qR': -0.33,
    'qE': -0.33,
    'hpRE': 0.1,
    'hpER': 0.3,
    'PE': 0.2,
    'PHL': 10.0
}

diffs = {}
for k in ref_params.keys():
    if k in summary.index:
        diffs[k] = summary.loc[k, 'mean'] - ref_params[k]

pd.Series(diffs).round(3).to_frame('Posterior - Reference').style_background_gradient(cmap='RdYlBu_r') 

In [ ]:
import datetime 

# Ensure output folder exists
summary_dir = "../data/results/summary"
os.makedirs(summary_dir, exist_ok=True) 

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
summary_csv = os.path.join(summary_dir, f"posterior_summary_{timestamp}.csv")
trace_png   = os.path.join(summary_dir, f"traceplot_{timestamp}.png")
posterior_png = os.path.join(summary_dir, f"posterior_density_{timestamp}.png")
diffs_csv   = os.path.join(summary_dir, f"posterior_minus_ref_{timestamp}.csv")

# Save posterior summary table
summary.round(4).to_csv(summary_csv)
print(f"Posterior summary saved to {summary_csv}")


fig_trace = az.plot_trace(idata, var_names=["hFEV1R", "hPROR", "r2_FEV1", "hpRE"], compact=True)
plt.tight_layout()
plt.savefig(trace_png, dpi=300)
plt.close()
print(f"Traceplot saved to {trace_png}")

fig_post = az.plot_posterior(idata, var_names=["hFEV1R", "hPROR", "hpRE", "hpER"], hdi_prob=0.95)
plt.tight_layout()
plt.savefig(posterior_png, dpi=300)
plt.close()
print(f"Posterior density plot saved to {posterior_png}")

# Save difference (Posterior − Reference) table
pd.Series(diffs).round(4).to_csv(diffs_csv)
print(f"Differences vs reference saved to {diffs_csv}")


## Summary 

- In this notebook we have:

1. Implemented two estimation approaches for the mixed Hidden Markov Model (mHMM):
   - **Monte Carlo–based Maximum Likelihood Estimation (MLE)** for quick testing and validation.
   - **Bayesian estimation in Stan (CmdStanPy)** as an open-source alternative to NONMEM + SAEM.

2. Utilized the Stan model which reproduces the same likelihood structure as the paper:
   - Population modes (`h*`), individual random effects (`g`), and residual variances (`r2_*`).
   - State-specific correlations and time-dependent placebo term (`PE`, `PHL`).
   - Transition probabilities following logistic forms from Eqs. 7–10.

3. We flattened the simulated data from `02_simulator.ipynb` and formatted for Stan’s input requirements.

4. Provided code to summarize sampling results, visualize (trace and posterior densities), and compare to the reference scenario (Table 1 in the paper).

5. Exported all posterior summaries and diagnostics to `data/results/summary/` for reproducibility and later use in the simulation–estimation evaluation (SSE) stage.
